In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

import geopandas as gpd
from shapely.geometry import Point, Polygon

In [2]:
try:
  %load_ext autotimme
except:
  !pip install ipython-autotime
  %load_ext autotime
print("ok")

ok
time: 1.64 ms (started: 2024-12-04 03:33:35 +00:00)


In [3]:
#!pip install psycopg2-binary sqlalchemy pymysql python-dotenv geopandas shapely 

time: 8.15 ms (started: 2024-12-04 03:33:44 +00:00)


In [4]:
# Carregar variáveis do arquivo .env
load_dotenv()

# Recuperar as variáveis do banco de dados
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

# Exibir as variáveis (opcional, para verificação)
print(f"Conectando ao banco {database} em {host}:{port} com o usuário {user}")

Conectando ao banco ada_tech em 129.148.25.96:3306 com o usuário alunos_ada
time: 15.7 ms (started: 2024-12-04 03:33:47 +00:00)


In [5]:
# Criar a engine de conexão
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

time: 155 ms (started: 2024-12-04 03:33:48 +00:00)


In [6]:
# # Configurações do banco de dados
# host = "129.148.25.96"
# port = 3306 
# user = "alunos_ada"
# password = "i#C1C3lP3HX5wckE"
# database = "ada_tech"

# # String de conexão para o SQLAlchemy
# engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

time: 5.02 ms (started: 2024-12-04 03:33:52 +00:00)


## Consulta 1

In [7]:
# /* Consulta 1
# 	Recupere a soma de área (em hectares) para todas as 
# 	propriedades agrícolas que pertencem ao MS e MT. 
# 	Ordene os resultados em ordem decrescente.  */

# Query SQL
query = """select 
        	sum(area_do_imovel) as soma_area_imovel 
            from ada_tech.prof_car_local 
            where uf='MS' or uf='MT' 
            group by uf
            order by soma_area_imovel desc;"""

# Carregar os dados em um DataFrame
df1 = pd.read_sql(query, con=engine)

# Exibir os dados
df1

,soma_area_imovel
0,8.458709e+07
1,3.632188e+07


time: 4.75 s (started: 2024-12-04 03:33:54 +00:00)


## Consulta 2

In [16]:
# /* Consulta 2
# 	Filtre todas as propriedades que pertecem a região sudeste. */

# Query SQL
query = """select distinct 
        	uf
        	,municipio
        	,codigo_ibge
        	,area_do_imovel
        	,registro_car 
        from ada_tech.prof_car_local 
        where uf in ('SP','MG','ES','RJ');"""

# Carregar os dados em um DataFrame
df2 = pd.read_sql(query, con=engine)

# Exibir os dados
df2

,uf,municipio,codigo_ibge,area_do_imovel,registro_car
0,RJ,Itaperuna,3302205,6.0851,RJ-3302205-169354AD04A446EC9AB27A79E4310489
1,RJ,Itaperuna,3302205,44.4215,RJ-3302205-1DEA0210F20F4ECEBADF8585534E9177
2,RJ,Itaperuna,3302205,0.4112,RJ-3302205-8ACBE3784F13410A9835E85EB743E95B
3,RJ,Rio Bonito,3304300,93.9777,RJ-3304300-66B9CE428807488391B38ACFBDCF1DBD
4,MG,Sabará,3156700,2.3900,MG-3156700-A61D7CC1D7994BFE939ACC2D69CCA87A
...,...,...,...,...,...
1564448,MG,Caratinga,3113404,3.9604,MG-3113404-43859270A5D54EE98F8462FC7C510C34
1564449,MG,Pedras de Maria da Cruz,3149150,0.2445,MG-3149150-E8AB58E101F943DF97CFDFD09D9003B1
1564450,MG,Tapira,3168101,6.4957,MG-3168101-1F694940FC9940D0ADBF8D1DE180DE69
1564451,MG,Dom Silvério,3122702,0.1187,MG-3122702-4C960E6B2CF5450FB49452DC39773884


time: 30.2 s (started: 2024-12-04 03:07:30 +00:00)


## Consulta 3

In [ ]:
# Consulta 3
# Recupere todas as propriedades rurais que estão localizadas 
# dentro de uma área geográfica específica delimitada por um polígono. 
# Este polígono é descrito pelas seguintes coordenadas: 
# POLYGON ((-53.5325072 -19.4632582, -51.0495971 -19.1625841, -51.3734501 -16.1924262, -53.8181518 -16.4010783, -53.5325072 -19.4632582))



# Query SQL
query = """select distinct
            	count(pcl.registro_car)
            from ada_tech.prof_car_local pcl 
            where pcl.tipo_imovel_rural = 'IRU';"""

# Carregar os dados em um DataFrame
df3 = pd.read_sql(query, con=engine)

# Exibir os dados
df3

In [ ]:
# Coordenadas para o polígono
coords = [
    (-53.5325072, -19.4632582),
    (-51.0495971, -19.1625841),
    (-51.3734501, -16.1924262),
    (-53.8181518, -16.4010783),
    (-53.5325072, -19.4632582)
]

# Criar o polígono
polygon = Polygon(coords)

# Exibir o polígono (opcional)
print(f"Polígono criado: {polygon}")

# Dados da base (exemplo)
data = {
    "id": [1, 2, 3, 4],
    "latitude": [-19.4, -17.0, -18.5, -19.5],
    "longitude": [-53.0, -51.5, -52.0, -53.8]
}
df = pd.DataFrame(data)

# Criar um GeoDataFrame com os pontos da base
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"
)

# Verificar se os pontos estão dentro do polígono
gdf["inside_polygon"] = gdf.geometry.apply(lambda point: polygon.contains(point))

# Filtrar apenas os pontos dentro do polígono
inside_points = gdf[gdf["inside_polygon"]]

# Exibir os resultados
print("Pontos dentro do polígono:")
print(inside_points)


## Consulta 4 

In [17]:
# /* Consulta 4
# 	Calcule quantas propriedades foram cadastradas por ano.
# 	Apresente os resultados em ordem cronológica. */

# Query SQL
query = """select distinct
            	year(data_inscricao) as ano
            	,count(registro_car) as propriedade_cadastradas
            from ada_tech.prof_car_local 
            group by ano;"""

# Carregar os dados em um DataFrame
df4 = pd.read_sql(query, con=engine)

# Exibir os dados
df4

,ano,propriedade_cadastradas
0,2014.0,227468
1,2015.0,1240178
2,NaN,12
3,2016.0,1828786
4,2013.0,743
5,2017.0,794779
6,2018.0,724856
7,2019.0,746570
8,2020.0,463322
9,2021.0,443942


time: 12.2 s (started: 2024-12-04 03:08:24 +00:00)


## Consulta 5

In [11]:
# # /* Consulta 5
# # 	Calcule o percentual médio de área remanescente de 
# # 	vegetação nativa em comparação a área total da propriedade */

# # Query SQL
# query = """select 
#         	pcl.uf 
#         	,pcl.municipio 
#         	,pcl.registro_car 
#         	,pcl.area_do_imovel 
#         	,(pcl.area_remanescente_vegetacao_nativa / pcl.area_liquida) * 100 as pct_medio_area_vegetacao
#         from ada_tech.prof_car_local pcl;"""

# # Carregar os dados em um DataFrame
# df5 = pd.read_sql(query, con=engine)

# # Exibir os dados
# df5.head(10)

time: 636 µs (started: 2024-12-04 03:27:32 +00:00)


## Consulta 6

In [6]:
# /* Consulta 6
# 	Construa uma consulta que mostre a contagem de propriedades rurais por estado. */

# Query SQL
query = """select distinct
            	pcl.uf
            	,count(pcl.tipo_imovel_rural) as qnt_propriedades_rurais
            from ada_tech.prof_car_local pcl 
            where pcl.tipo_imovel_rural = 'IRU'
            group by pcl.uf;"""

# Carregar os dados em um DataFrame
df6 = pd.read_sql(query, con=engine)

# Exibir os dados
df6

,uf,qnt_propriedades_rurais
0,GO,198524
1,SC,374768
2,PR,495650
3,RJ,58848
4,MA,272010
5,RS,606014
6,PI,254577
7,PE,338392
8,DF,17335
9,MG,988923


time: 11.1 s (started: 2024-12-04 03:23:11 +00:00)


## Consulta 7

In [8]:
# /* Consulta 7
# 	Veja qual é a maior propriedade entre todas e calcule a distância 
# 	entre ela e Brasília. Utilize a coordenada de centródide da propriedade 
# 	para calcular a distância entre ela e Brasília. 
# 	Coordenadas de Brasília: -15.796943053171708, -47.891638482569476 */

# Query SQL
query = """select
            	pcl.registro_car 
            	,pcl.area_do_imovel 
            	,pcl.latitude
            	,pcl.longitude
            	,(6371 * ACOS(
                    COS(RADIANS(-15.796943053171708)) * COS(RADIANS(pcl.latitude)) *
                    COS(RADIANS(pcl.longitude) - RADIANS(-47.891638482569476)) +
                    SIN(RADIANS(-15.796943053171708)) * SIN(RADIANS(pcl.latitude))
                )) AS distancia_km
            from ada_tech.prof_car_local pcl 
            where pcl.registro_car = (select distinct
            							registro_car 
            						from ada_tech.prof_car_local
            						group by registro_car 
            						order by max(area_do_imovel) DESC limit 1);"""

# Carregar os dados em um DataFrame
df7 = pd.read_sql(query, con=engine)

# Exibir os dados
df7

,registro_car,area_do_imovel,latitude,longitude,distancia_km
0,AM-1302306-C937E84784BA47C3A8B97A5B3FA2727D,2.420079e+06,-5.475619,-68.888188,2562.129615


time: 1min 13s (started: 2024-12-04 03:24:11 +00:00)


## Consulta 8

In [9]:
# /* Consulta 8
# 	Faça a média de área entre todas as propriedades. Calcule quantas 
# 	propriedades por estado, estão acima da média. */

# Query SQL
query = """select distinct 
            	uf
            	,count(pcl.registro_car) as qnt_propriedades
            	,avg(pcl.area_do_imovel) as media_area
            from ada_tech.prof_car_local pcl
            group by uf
            having media_area > (select avg(area_do_imovel) from ada_tech.prof_car_local);"""

# Carregar os dados em um DataFrame
df8 = pd.read_sql(query, con=engine)

# Exibir os dados
df8

,uf,qnt_propriedades,media_area
0,GO,199690,164.781607
1,MA,273642,112.599726
2,AM,73642,968.969346
3,RR,22687,390.411597
4,AP,10116,511.920164
5,AC,45141,306.883235
6,TO,85537,293.615894
7,MT,170205,496.971824
8,RO,147287,108.260902
9,PA,279598,293.775381


time: 20 s (started: 2024-12-04 03:26:06 +00:00)


In [5]:
# Host: 129.148.25.96
# Port: 3306
# User: alunos_ada
# Schema: ada_tech
# Password: i#C1C3lP3HX5wckE

In [4]:
import pandas as pd

# Caminho para o arquivo CSV
csv_path = "/home/jovyan/app/data/temas_ambientais.csv"  # Adapte para o local correto no container

# Nome da tabela no PostgreSQL
table_name = "minha_tabela"

# Lê o CSV e insere em chunks
chunk_size = 10000  # Número de linhas por chunk
for chunk in pd.read_csv(csv_path, sep=';', chunksize=chunk_size):
    # Insere no banco de dados
    chunk.to_sql(table_name, engine, if_exists="append", index=False)
    print(f"{len(chunk)} linhas inseridas na tabela {table_name}.")


10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.
10000 linhas inseridas na tabela minha_tabela.


ParserError: Error tokenizing data. C error: Expected 2 fields in line 205217, saw 3
